In [1]:
import numpy as np
import pandas as pd
from datetime import datetime as dt
import re
import sys
import itertools
import MeCab
import matplotlib.pyplot as plt
import MeCab
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

import collections
import itertools

import scipy as sp

In [35]:
df1 = pd.read_excel('./PatentPositiveWords.xlsx')
# df2 = pd.read_excel('./PatentsPositiveWords_terms_all.xlsx')
# df3 = pd.read_excel('./PatentsPositiveWords_terms_summary.xlsx')

In [36]:
df1.head()

,Unnamed: 0,公開・公表番号,発明の名称,公開・公表日,出願日,原出願日,出願人・権利者名,出願番号,登録番号,筆頭発明者,...,all_延設,all_延材,all_狭持,all_回転方向,summary_延設,summary_延材,summary_狭持,summary_回転方向,all_dic,summary_dic
0,0,登実第3082540号,Ｚ形仕切付段ボール箱,2001.12.14,2001-06-08,NaT,佐藤工業株式会社,実願2001-003729,登実第3082540号,佐藤 浩康,...,0,0,0,0,0,0,0,0,"{'連設': 10, '段ボールシート': 5, '仕切り壁': 3, '箱本体': 3, ...","{'一対': 2, '連設': 2}"
1,1,登実第3081641号,吊りフック,2001.11.16,2001-05-10,NaT,"寄神建設株式会社,東亜建設工業株式会社,社団法人日本海上起重技術協会",実願2001-002834,登実第3081641号,石塚 浩次,...,0,0,0,0,0,0,0,0,"{'玉掛けワイヤ': 14, 'フックピン': 13, 'メッセンジャーワイヤ': 10, ...","{'下げ状態': 2, '回動': 2, '下げ支持': 1, '遠隔操作': 1}"
2,2,特開2001-311113,橋脚柱の構築工法,2001.11.09,2001-04-19,1995-06-30,株式会社大林組,特願2001-120638,NaN,大内 一,...,0,0,0,0,0,0,0,0,"{'構築工法': 19, 'フーチング部': 7, '作業足場': 7, 'コンクリート打設...","{'構築工法': 3, 'コンクリート打設工程': 1, '作業足場': 1, '揚重機':..."
3,3,特開2001-355221,コンクリートの運搬装置,2001.12.26,2001-04-06,1997-07-24,株式会社大林組,特願2001-109004,NaN,藤原 宗一,...,2,0,0,0,1,0,0,0,"{'運搬装置': 20, '螺旋羽根': 14, 'コンクリートプラント': 10, '回転...","{'螺旋状空間': 4, '螺旋羽根': 3, 'コンクリートプラント': 1, 'シュート..."
4,4,特開2001-348842,コンクリートの運搬方法,2001.12.21,2001-04-06,1997-07-24,株式会社大林組,特願2001-108995,NaN,藤原 宗一,...,2,0,0,0,1,0,0,0,"{'螺旋羽根': 11, 'コンクリートプラント': 9, '材料分離': 7, '回転駆動...","{'螺旋状空間': 4, '螺旋羽根': 3, 'コンクリートプラント': 1, 'シュート..."


In [7]:
df1.columns

Index(['Unnamed: 0', '公開・公表番号', '発明の名称', '公開・公表日', '出願日', '原出願日', '出願人・権利者名',
       '出願番号', '登録番号', '筆頭発明者', '発明者名', 'IPC', '筆頭ＩＰＣ', 'Ｆターム', 'FI', 'ステイタス',
       'ST', '早期請求出願有無', '要約', '全請求項', '詳細な説明', 'number', 'company',
       'datetime', 'all_延設', 'all_延材', 'all_狭持', 'all_回転方向', 'summary_延設',
       'summary_延材', 'summary_狭持', 'summary_回転方向', 'all_dic', 'summary_dic'],
      dtype='object')

In [37]:
df_q = df1[ (df1['all_延設'] > 0) | (df1['all_延材'] > 0) |(df1['all_狭持'] > 0) |(df1['all_狭持'] > 0)]

In [38]:
df = df_q.drop(columns=['Unnamed: 0', '公開・公表日', '出願日', '原出願日', '出願人・権利者名',
       '出願番号', '登録番号', 'FI', 'ステイタス', 'number',  
       'ST', '早期請求出願有無', '要約', '全請求項', '詳細な説明', 'all_延設', 'all_延材', 'all_狭持', 'all_回転方向', 'summary_延設',
       'summary_延材', 'summary_狭持', 'summary_回転方向'])
# df = df.drop(columns=['number', '発明者名	'])
#  columns=['Unnamed: 0','早期請求出願有無', '要約', '全請求項', '詳細な説明', 'ST', 'ステイタス', '原出願日', '出願日', '出願日', '登録番号', '公開・公表番号', 'sentence', '出願人・権利者名', '公開・公表日']


In [39]:
df.head()

,公開・公表番号,発明の名称,筆頭発明者,発明者名,IPC,筆頭ＩＰＣ,Ｆターム,company,datetime,all_dic,summary_dic
3,特開2001-355221,コンクリートの運搬装置,藤原 宗一,藤原 宗一,"E02B 7/00 (2006.01),B65G 33/12 (2006.01)",E02B 7/00 (2006.01),"3F040BA01,3F040CA04,3F040EA03",{'株式会社大林組'},2001-12-26,"{'運搬装置': 20, '螺旋羽根': 14, 'コンクリートプラント': 10, '回転...","{'螺旋状空間': 4, '螺旋羽根': 3, 'コンクリートプラント': 1, 'シュート..."
4,特開2001-348842,コンクリートの運搬方法,藤原 宗一,藤原 宗一,E02B 7/00 (2006.01),E02B 7/00 (2006.01),NaN,{'株式会社大林組'},2001-12-21,"{'螺旋羽根': 11, 'コンクリートプラント': 9, '材料分離': 7, '回転駆動...","{'螺旋状空間': 4, '螺旋羽根': 3, 'コンクリートプラント': 1, 'シュート..."
5,特開2001-348844,ダム用コンクリートの運搬方法,藤原 宗一,藤原 宗一,"E02B 7/00 (2006.01),B65G 33/12 (2006.01)",E02B 7/00 (2006.01),"3F040BA01,3F040EA03,3F040FA03,3F040FA05",{'株式会社大林組'},2001-12-21,"{'ダム用コンクリート': 15, '螺旋羽根': 11, 'コンクリートプラント': 8,...","{'螺旋状空間': 4, '螺旋羽根': 3, 'コンクリートプラント': 1, 'シュート..."
6,特開2001-348843,コンクリートの運搬方法,藤原 宗一,藤原 宗一,"E02B 7/00 (2006.01),B65G 11/06 (2006.01)",E02B 7/00 (2006.01),"3F011AA02,3F011BA01,3F011BC07",{'株式会社大林組'},2001-12-21,"{'コンクリートプラント': 10, '材料分離': 7, 'ダム用コンクリート': 4, ...","{'コンクリートプラント': 1, '外周縁': 1, '実施例': 1, '屈曲形成': ..."
43,特開2001-317289,シールド掘進機,小林 信明,"小林 信明,戸田 浩,小林 弘太郎","E21D 9/06 (2006.01),E21D 11/40 (2006.01)",E21D 9/06 (2006.01),"2D054AC04,2D054AC05,2D054AD13,2D054AD19,2D054A...",{'大成建設株式会社'},2001-11-16,"{'鋼殻': 36, 'トンネル掘削': 13, '排土機構': 12, '掘削機構': 1...","{'シールドジャッキ': 1, '補助工法': 1, '進退自在': 1, '鋼殻': 1,..."


In [40]:
df = df.rename(columns={'発明の名称': 'patent_name', '筆頭ＩＰＣ': 'headIPC', '出願番号': 'id', 'Ｆターム': 'f_term', 'company': 'createdBy', 'datetime':'createdAt', 'terms':'term', '発明者名':'inventor', '筆頭発明者':'head_inventor', '公開・公表番号': 'patent_id'})

In [41]:
df.head()

,patent_id,patent_name,head_inventor,inventor,IPC,headIPC,f_term,createdBy,createdAt,all_dic,summary_dic
3,特開2001-355221,コンクリートの運搬装置,藤原 宗一,藤原 宗一,"E02B 7/00 (2006.01),B65G 33/12 (2006.01)",E02B 7/00 (2006.01),"3F040BA01,3F040CA04,3F040EA03",{'株式会社大林組'},2001-12-26,"{'運搬装置': 20, '螺旋羽根': 14, 'コンクリートプラント': 10, '回転...","{'螺旋状空間': 4, '螺旋羽根': 3, 'コンクリートプラント': 1, 'シュート..."
4,特開2001-348842,コンクリートの運搬方法,藤原 宗一,藤原 宗一,E02B 7/00 (2006.01),E02B 7/00 (2006.01),NaN,{'株式会社大林組'},2001-12-21,"{'螺旋羽根': 11, 'コンクリートプラント': 9, '材料分離': 7, '回転駆動...","{'螺旋状空間': 4, '螺旋羽根': 3, 'コンクリートプラント': 1, 'シュート..."
5,特開2001-348844,ダム用コンクリートの運搬方法,藤原 宗一,藤原 宗一,"E02B 7/00 (2006.01),B65G 33/12 (2006.01)",E02B 7/00 (2006.01),"3F040BA01,3F040EA03,3F040FA03,3F040FA05",{'株式会社大林組'},2001-12-21,"{'ダム用コンクリート': 15, '螺旋羽根': 11, 'コンクリートプラント': 8,...","{'螺旋状空間': 4, '螺旋羽根': 3, 'コンクリートプラント': 1, 'シュート..."
6,特開2001-348843,コンクリートの運搬方法,藤原 宗一,藤原 宗一,"E02B 7/00 (2006.01),B65G 11/06 (2006.01)",E02B 7/00 (2006.01),"3F011AA02,3F011BA01,3F011BC07",{'株式会社大林組'},2001-12-21,"{'コンクリートプラント': 10, '材料分離': 7, 'ダム用コンクリート': 4, ...","{'コンクリートプラント': 1, '外周縁': 1, '実施例': 1, '屈曲形成': ..."
43,特開2001-317289,シールド掘進機,小林 信明,"小林 信明,戸田 浩,小林 弘太郎","E21D 9/06 (2006.01),E21D 11/40 (2006.01)",E21D 9/06 (2006.01),"2D054AC04,2D054AC05,2D054AD13,2D054AD19,2D054A...",{'大成建設株式会社'},2001-11-16,"{'鋼殻': 36, 'トンネル掘削': 13, '排土機構': 12, '掘削機構': 1...","{'シールドジャッキ': 1, '補助工法': 1, '進退自在': 1, '鋼殻': 1,..."


In [42]:
ret = df.to_dict(orient='index')
df['createdAt'] = df['createdAt'].astype(str)
print (df.dtypes)

patent_id        object
patent_name      object
head_inventor    object
inventor         object
IPC              object
headIPC          object
f_term           object
createdBy        object
createdAt        object
all_dic          object
summary_dic      object
dtype: object


In [43]:
df.reset_index(drop=True, inplace=True)
path = './PatentPositiveWords.json'
df.to_json(path, force_ascii=False, indent=2, orient='index')

In [31]:
df2 = pd.read_excel('./PatentsPositiveWords_terms_all.xlsx')
df3 = pd.read_excel('./PatentsPositiveWords_terms_summary.xlsx')

In [33]:
df2

,Unnamed: 0,terms,document_frequency,max_term_frequency,sumof_term_frequency
0,90695,延設,2586,91,10406
1,51391,有効利用,1953,55,4307
2,1409,アンカーボルト,1939,204,17780
3,54412,横断面,1919,40,5505
4,44446,技術的思想,1909,11,2351
...,...,...,...,...,...
90694,72784,自家消費,5,4,18
90695,46446,採光パネル,5,38,48
90696,46447,採光効率,5,5,13
90697,46453,採光環境,5,4,11


In [34]:
df3

,Unnamed: 0,terms,document_frequency,max_term_frequency,sumof_term_frequency
0,20312,配設,2077,6,2740
1,13596,水平方向,1427,9,1945
2,3369,上下方向,1258,9,1647
3,20793,長手方向,984,6,1334
4,19509,軸方向,912,10,1306
...,...,...,...,...,...
21969,11739,新設屋根,3,5,9
21970,16607,管状態,3,2,4
21971,3167,一体化強度,3,5,8
21972,0,γ線検出,3,3,6
